In [262]:
import sympy as sp
from IPython.display import Math

# Utility functions

In [263]:
def display_general_solution_latex_factored(sols, domain='real', x_symbol='x', symbol_index=None):
    """
    Display general solution of a linear system of ODEs with symbolic constants and
    factored common multiplicative terms (e.g., exp(t)).

    Parameters:
        sols        : SymPy Matrix where each column is a solution vector
        domain      : 'real', 'complex', or None — determines cᵢ ∈ ℝ or ℂ, or omits constants
        x_symbol    : function name for the solution vector (default: 'x')
        symbol_index: optional index for subscripting x_symbol
    """
    terms = []
    constants = []

    for idx in range(sols.shape[1]):
        sol = sols[:, idx]
        
        if domain is not None:
            c = sp.symbols(f'c{idx + 1}', real=(domain == 'real'))
            constants.append(c)
        else:
            c = None

        flat_entries = list(sol)
        factor = sp.gcd_list(flat_entries)

        if factor == 1:
            term = sp.latex(sol)
            if c is not None:
                term_latex = f"{sp.latex(c)} {term}"
            else:
                term_latex = term
        else:
            simplified_vec = sp.simplify(sol / factor)
            term = f"{sp.latex(factor)} {sp.latex(simplified_vec)}"
            if c is not None:
                term_latex = f"{sp.latex(c)} {term}"
            else:
                term_latex = term

        terms.append(term_latex)

    # Construct solution expression
    prefix = rf"{x_symbol}_{{{symbol_index}}}(t)" if symbol_index else rf"{x_symbol}(t)"
    latex_expr = f"{prefix} = " + " + ".join(terms)

    if domain is not None and constants:
        constant_list = ", ".join(sp.latex(c) for c in constants)
        number_set = r"\mathbb{R}" if domain == 'real' else r"\mathbb{C}"
        latex_expr += rf", \quad \text{{where }} {constant_list} \in {number_set}"

    display(Math(latex_expr))


# Systems of 1st order differential equations

In [405]:

# Define the matrix
A = sp.Matrix([[2, 9], [-1, -4]]) # Example matrix with 2 real eigenvalues
A = sp.Matrix([[0, 0, 1], [2, 0, -1], [-1, 0, 0]]) # Example matrix with 1 real and 2 complex eigenvalues
A = sp.Matrix([[2, 9], [-1, -4]])
A = sp.Matrix([[1, 2], [2, 4]])
t = sp.symbols('t', real=True)
A

Matrix([
[1, 2],
[2, 4]])

### Homogenous system

#### General real solution

In [380]:
def construct_real_solutions(A, tau_vals=None, verbose=True):
    """
    Construct the fundamental matrix solution Φ(t) to the system x'(t) = A x(t),
    using:
      - Theorem 2.7(a) for simple real eigenvalues
      - Theorem 2.7(b) for simple complex eigenvalues
      - Theorem 2.11 (special case) for eigenvalues with algebraic multiplicity 2 and geometric multiplicity 1
    """
    t = sp.symbols('t', real=True)
    I_n = sp.eye(A.shape[0])
    cols = []
    if verbose:
        display(Math(r"\text{Constructing fundamental matrix } \Phi(t) \text{ for the system } x'(t) = A x(t)"))
        display(Math(r"A = " + sp.latex(A)))
        char_poly = A.charpoly()
        display(Math(r"\text{Characteristic polynomial: } " + sp.latex(char_poly)))
        display(Math(r"\text{Characteristic polynomial factored: } " + sp.latex(char_poly.as_expr().factor())))

    if verbose:
      print("Iterating over eigenvalues and eigenvectors:")
    for eigenvalue, algebraic_mult, eigenvectors in A.eigenvects():
        geometric_mult = len(eigenvectors)

        if eigenvalue.is_real:
            if algebraic_mult == 1:
                # Theorem 2.7(a)
                v = eigenvectors[0]
                if verbose:
                    print(f"Eigenvalue = {eigenvalue}. Algebraic multiplicity = 1. Geometric multiplicity = 1. Using Theorem 2.7(a)")
                cols.append(sp.exp(eigenvalue * t) * v)

            elif algebraic_mult == 2 and geometric_mult == 1:
                # Theorem 2.11(a)
                if verbose:
                    print(f"Eigenvalue = {eigenvalue}. Eigenvector = {eigenvectors[0]}. Algebraic multiplicity = 2. Geometric multiplicity = 1. Using Theorem 2.11(a)")

                b11 = eigenvectors[0]
                J = A - eigenvalue * I_n
                b21, tau = J.gauss_jordan_solve(b11)

                # Set all free parameters
                if tau_vals is not None:
                    b21 = b21.subs({s: tau_vals for s in tau.free_symbols})

                x1 = sp.exp(eigenvalue * t) * b21
                x2 = sp.exp(eigenvalue * t) * (b21 + t * b11)

                if verbose:
                  display(Math(r"J = A - \lambda I = " + sp.latex(J)))
                  
                  # Display solutions
                  display(Math(fr"b_{{11}} = {sp.latex(b11)}"))

                  display(Math(r"\text{find } b_{ 21 } \text{ by solving } J b_{ 21 } = b_{22} \text{ for } b_{ 22 } = b_{ 11 }"))
                  display(Math(fr"b_{{21}} = {sp.latex(b21)}"))
                  display(Math(fr"b_{{22}} = {sp.latex(b11)}"))
                  display(Math(fr"x_{{1}} = {sp.latex(x1)}"))
                  display(Math(fr"x_{{2}} = {sp.latex(x2)}"))

                cols.extend([x1, x2])

        else:
            if sp.im(eigenvalue) < 0:
                continue  # Skip complex conjugate

            v = eigenvectors[0]
            J = A - eigenvalue * I_n

            if algebraic_mult == 1:
                # Theorem 2.7(b)
                complex_sol = sp.exp(eigenvalue * t) * v
                x1 = sp.re(complex_sol)
                x2 = sp.im(complex_sol)

                if verbose:
                    print(f"Eigenvalue = {eigenvalue}. Eigenvector = {v}. Algebraic multiplicity = 1. Geometric multiplicity = 1. Using Theorem 2.7(b)")
                    display(Math(fr"complex \, solution = {sp.latex(complex_sol)}"))
      
                    display(Math(fr"x_{{1}} = {sp.latex(x1)}"))
                    display(Math(fr"x_{{2}} = {sp.latex(x2)}"))

                cols.extend([x1, x2])

            elif algebraic_mult == 2 and geometric_mult == 1:
                # Extended complex eigenvalue case
                b2 = v
                b1, tau = J.gauss_jordan_solve(b2)

                if tau_vals is not None:
                    b1 = b1.subs({s: tau_vals for s in tau.free_symbols})
                else:
                    b1 = b1.subs({s: 0 for s in tau.free_symbols})

                x1 = sp.exp(eigenvalue * t) * b2
                x2 = sp.exp(eigenvalue * t) * (b1 + t * b2)

                real_x1 = sp.re(x1)
                imag_x1 = sp.im(x1)
                real_x2 = sp.re(x2)
                imag_x2 = sp.im(x2)

                if verbose:
                    print(f"Eigenvalue = {eigenvalue}. Algebraic multiplicity = 2. Geometric multiplicity = 1. Using extended Theorem 2.7(b)")

                cols.extend([real_x1, imag_x1, real_x2, imag_x2])

    fundamental_matrix = sp.Matrix.hstack(*cols)

    if verbose:
        display(Math(r"\Phi(t) = " + sp.latex(fundamental_matrix)))
        display_general_solution_latex_factored(fundamental_matrix, domain='real', x_symbol='x')

    return fundamental_matrix, t


In [406]:
sols = construct_real_solutions(A, verbose = True , tau_vals = 1)
sols

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Iterating over eigenvalues and eigenvectors:
Eigenvalue = 0. Algebraic multiplicity = 1. Geometric multiplicity = 1. Using Theorem 2.7(a)
Eigenvalue = 5. Algebraic multiplicity = 1. Geometric multiplicity = 1. Using Theorem 2.7(a)


<IPython.core.display.Math object>

<IPython.core.display.Math object>

(Matrix([
 [-2, exp(5*t)/2],
 [ 1,   exp(5*t)]]),
 t)

### Inhomogeneous system

In [442]:

import dis


A = sp.Matrix([[1, 2], [2, 4]])
display(Math(r"A = " + sp.latex(A)))
u = sp.Matrix([[1],[1]])
display(Math(r"u = " + sp.latex(u)))
# display find general real solution for the system
display(Math(r"\text{Find general real solution for the system } x'(t) = A x(t) + u"))

display(Math(r"\text{Constructing fundamental matrix } \Phi(t) \text{ for the homogenous system } x'(t) = A x(t)"))
fundamental_matrix, t = construct_real_solutions(A, verbose = False, tau_vals = None)
c_vec = sp.Matrix([sp.symbols(f'c{idx + 1}', real=True) for idx in range(sols[0].shape[1])])
t0 = sp.symbols('t0', real=True)


display(Math(r"\Phi(t) = " + sp.latex(fundamental_matrix)))
inv_fundamental_matrix = fundamental_matrix.inv()
display(Math(r"\Phi(t)^{-1} = " + sp.latex(inv_fundamental_matrix)))
inv_fundamental_matrix_u = inv_fundamental_matrix * u
display(Math(r"\Phi(t)^{-1} u = " + sp.latex(inv_fundamental_matrix_u)))
integral = sp.integrate(inv_fundamental_matrix_u, (t, t0, t))
display(Math(r"\int_{t_0}^{t} \Phi(t)^{-1} \bold{u} \, dt = " + sp.latex(integral)))
fundamental_matrix_integral = fundamental_matrix * integral
display(Math(r"\Phi(t) \int_{t_0}^{t} \Phi(t)^{-1} \bold{u} \, dt = " + sp.latex(fundamental_matrix_integral)))
display(Math(r"\text{General solution: } x(t) = \Phi(t) \bold{c}  + \Phi(t) \int_{t_0}^{t} \Phi(t)^{-1} \bold{u} \, dt \, \text{ where } \bold{c} \in \mathbb{R}^{" + str(c_vec.shape[0]) + "}"))
general_solution = fundamental_matrix * c_vec + fundamental_matrix_integral
display(Math(r"\bold{x}(t) = " + sp.latex(general_solution.simplify())))
# term_1 = fundamental_matrix * c_vec
# term_2 = fundamental_matrix * integral

# term_2



<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>